In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from ase.io import read
from matscipy.angle_distribution import angle_distribution
from matscipy.neighbours import neighbour_list

def calculate_adf(filepath, cutoff, nbins, sigma_deg):
    atoms = read(filepath)
    i, j, d = neighbour_list('ijD', atoms, cutoff=cutoff)
    nb_angles = angle_distribution(i, j, d, nbins)
    bin_spacing_rad = np.pi / nbins
    angle_rad = (np.arange(nbins) + 0.5) * bin_spacing_rad
    angle_deg = np.degrees(angle_rad)
    total_angles = np.sum(nb_angles)
    probability = nb_angles / (total_angles * bin_spacing_rad)
    bin_width_deg = angle_deg[1] - angle_deg[0]
    sigma_in_bins = sigma_deg / bin_width_deg
    probability_smoothed = gaussian_filter1d(probability, sigma=sigma_in_bins)
    return angle_deg, probability_smoothed

files_to_process = [
    {'path': "a-As.extxyz", 'label': r'${a}$-As', 'color': '#cc5c5c', 'cutoff': 2.9, 'style': '-'},
    {'path': "a-P.extxyz", 'label': r'${a}$-P', 'color': '#7c87b5', 'cutoff': 2.4, 'style': '--'}
]

fig, ax = plt.subplots(figsize=(2.4, 2.4))
nbins = 180
smoothing_sigma = 2

for idx, f in enumerate(files_to_process):
    angle_deg, prob = calculate_adf(f['path'], f['cutoff'], nbins, smoothing_sigma)
    if f['style'] == '--':
        line, = ax.plot(angle_deg, prob, label=f['label'], color=f['color'], zorder=2)
        line.set_dashes([2, 1])  
    else:
        ax.plot(angle_deg, prob, label=f['label'], color=f['color'], linestyle=f['style'], zorder=4)

ax.set_xlabel("Bond angle (deg)")
ax.set_ylabel("Normalized distribution")
ax.set_xlim([30, 180])
ax.set_xticks(np.arange(30, 181, 30))
ax.set_ylim([-0.25, 5])
ax.set_yticks(np.arange(0, 5.1, 1))
plt.legend()

plt.show()